In [1]:
import os
os.chdir('..')

In [2]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [3]:
from tonal import pianoroll2number, joint_prob2pianoroll96
from tonal import tonal_centroid, chord482note, chord962note, note2number
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pickle
from model.surprisingness_CVAE import CVAE
from decode import *
import matplotlib.pyplot as plt

In [ ]:
melody_framewise = np.load('./data/melody_data.npy')
chord_groundtruth_idx = np.load('./data/chord_groundtruth.npy')

melody = np.load('./data/melody_baseline.npy')
chord = np.load('./data/number_96.npy')
chord_onehot = np.load('./data/onehot_96.npy')
length = np.load('./data/length.npy')

f = open('./data/tempos', 'rb')
tempos = pickle.load(f)
f.close()
f = open('./data/downbeats', 'rb')
downbeats = pickle.load(f)
f.close()

In [31]:
val_size = 500
print('splitting testing set...')
val_melody_framewise = melody_framewise[:val_size]
val_chord_groundtruth_idx = chord_groundtruth_idx[:val_size]

val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_melody = torch.from_numpy(melody[:val_size]).float()
val_length = torch.from_numpy(length[:val_size])


splitting testing set...


In [33]:
# Load model
device = 'cpu'
print('building model...')
model = CVAE(device = device).to(device)
model.load_state_dict(torch.load('output_models/model_surprisingness_cvae.pth'))
# model.load_state_dict(torch.load('output_models/model_ctdcvae_conv_prenet.pth'))
print(model)
model.eval()
val_length, val_melody = val_length.to(device), val_melody.to(device)

building model...


RuntimeError: Error(s) in loading state_dict for CVAE:
	size mismatch for encoder.weight_ih_l0: copying a param with shape torch.Size([2048, 96]) from checkpoint, the shape in current model is torch.Size([4096, 96]).
	size mismatch for encoder.weight_hh_l0: copying a param with shape torch.Size([2048, 512]) from checkpoint, the shape in current model is torch.Size([4096, 1024]).
	size mismatch for encoder.bias_ih_l0: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for encoder.bias_hh_l0: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for encoder.weight_ih_l0_reverse: copying a param with shape torch.Size([2048, 96]) from checkpoint, the shape in current model is torch.Size([4096, 96]).
	size mismatch for encoder.weight_hh_l0_reverse: copying a param with shape torch.Size([2048, 512]) from checkpoint, the shape in current model 

In [9]:
surp = np.load('./data/surprisingness.npy')

In [25]:
norm = np.max(surp)

In [28]:
## Profile function
x = np.arange(0,10,1) / norm
y = x

x = np.arange(10,0,-1) / norm
y = x


In [29]:
# Sampling
# torch.manual_seed(0)
i = 8
sample_melody = val_melody[i].unsqueeze(0)
sample_length = val_length[i]
sample_melody_framewise = np.expand_dims(val_melody_framewise[i],axis=0)
sample_chord_groundtruth_idx = np.expand_dims(val_chord_groundtruth_idx[i],axis=0)
sample_tempo = [tempos[i]]
sample_downbeat = [downbeats[i]]

## Surprising profile
s = [3,2,1,0,0,1,2,3]



melody_data[i] = np.pad(melody_data[i], ((0, max_melody_len-melody_data[i].shape[0]), (0, 0)), constant_values = (0, 0))

surprisingness = torch.Tensor([0] + s * 34).unsqueeze(0).unsqueeze(2)
surprisingness = model.surprisingness_prenet(surprisingness)
print(sample_length)

latent_size = 16

# for k in range(10):
latent = torch.randn(1,272,latent_size)
z = torch.cat([latent,sample_melody,surprisingness], dim=-1)
output, chord_pred = model.decode(z)

gen_chord_index = torch.max(chord_pred[0][:sample_length],-1).indices
print(gen_chord_index)
#     print(gen_chord_index.shape)

plt.subplot()


NameError: name 'model' is not defined

In [26]:
chord_pred.shape

torch.Size([1, 272, 96])

In [45]:
########## Random sampling ###########
# Proceed chord decode
print('proceed chord decode...')
joint_prob = chord_pred.cpu().detach().numpy()

# Append argmax index to get pianoroll array
accompany_pianoroll = argmax2pianoroll(joint_prob)

# augment chord into frame base
BEAT_RESOLUTION = 24
BEAT_PER_CHORD = 2

accompany_pianoroll_framewise, sample_chord_groundtruth_framewise = sequence2frame(accompany_pianoroll, sample_chord_groundtruth_idx)

# length into frame base
sample_length_framewise = sample_length * BEAT_RESOLUTION * BEAT_PER_CHORD

# write pianoroll
result_dir = 'results/surprising_all_cvae'
filename = str(i) + '-surprisingness-' + str(s)
write_one_pianoroll(result_dir, filename, sample_melody_framewise, accompany_pianoroll_framewise, sample_chord_groundtruth_framewise, sample_length_framewise, sample_tempo, sample_downbeat)


proceed chord decode...
accompany_pianoroll shape (1, 272, 128)
augment chord into frame base...
accompany_pianoroll frame shape: (1, 13056, 128)
groundtruth_pianoroll frame shape: (1, 13056, 128)
write pianoroll...
Finished!


In [37]:
sample_chord_groundtruth_framewise.shape

(1, 13056, 272, 128)